# pytorchvideo UFC101, pytorchvideo slowfast pretrain/scratch

pytorchvideonのdatasetを使ってUFC101を読み込み，pytorchvideoのslowfastモデルをfine-tuningしてみる．
UFC101はあらかじめダウンロードして展開済みであるとする．

- https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#ucf101

- https://pytorch.org/hub/facebookresearch_pytorchvideo_slowfast/



## ダウンロードできないというエラー

torchvisionをimportした後ではエラーが発生する（ImportError: cannot import name ***）

- https://github.com/pytorch/hub/issues/46


## 対応策

import torch直後に（import torchvisionをしない状態で）torch.hub.loadして，キャッシュに残しておく

こうすると，以降はキャッシュ（~/.cache/torch/hub/checkpoints/）が使われるのでエラーは発生しない

In [1]:
import torch
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

Using cache found in /home/tamaki/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DistributedSampler, RandomSampler


from torchvision import transforms


from pytorchvideo.models import x3d
from pytorchvideo.data import Ucf101, RandomClipSampler, UniformClipSampler


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
)


import torchinfo

from tqdm.notebook import tqdm
import itertools
import os
import pickle

argparseを真似たパラメータ設定．
- rootで指定したディレクトリには，101クラスのサブディレクトリがあること
- annotation_pathにはtrainlist0{1,2,3}.txtなどがあること

In [3]:
class Args:
    def __init__(self):
        self.metadata_path = '/mnt/HDD10TB/dataset/UFC101/'
        self.root = self.metadata_path + 'video/'
        self.annotation_path = self.metadata_path + 'ucfTrainTestlist/'
        self.frames_per_clip = 16
        self.step_between_clips = 16
        self.model = 'slowfast_r50'
        self.batch_size = 16
        self.num_workers = 24

        self.clip_duration = 16/25  # 25FPSを想定して16枚
        self.video_num_subsampled = 32  # 32枚抜き出す

args = Args()

transformの定義．
- UniformTemporalSubsampleで固定枚数をサンプルする
 - datasetのclip_samplerには，秒単位でしか与えられないようなので，fpsが異なる動画ではサンプルされる枚数も変わってくる．そのためここで取得するフレーム数を揃える（もっといい方法はないのか？）
- UCF101を読み込むとfloat32だが値は0-255，255で割ってfloatにする．
- slowfastを想定して，短い方を256画素程度に合わせてから，画像を256x256にリサイズする．
  - RandomShortSideScaleなら厳密には256にならない
  - ShortSideScaleなら256になる

バッチはdict形式なので，video, label, audioなどのそれぞれにtransformが設定できる
- ApplyTransformToKeyでkeyを指定して，video用のtransformを設定
- UCF101のラベルファイル（trainlist01.txtなど）には1から101までのラベルが付いているが，それがそのまま使われてしまうので（なぜだ．．．），このままではエラーが（不定期に）発生する．ラベルの値をtransformでから100にしておく
- audioは使わないのでRemoveKeyで除去

slowfastは，データをfast pathとして受け取って，それを時間方向にダウンサンプリングしたものをslow pathとして受け取る．
そのためにtransformでslow pathを作って，fast/slow のリストに変換する．
（slowfastのpretrainモデルは，このリストをデータとして受け取る）

In [4]:
# https://pytorch.org/hub/facebookresearch_pytorchvideo_slowfast/
num_frames = 32
sampling_rate = 2
frames_per_second = 25  # UCFは25と30が混在
clip_duration = (num_frames * sampling_rate)/frames_per_second
print(clip_duration)
side_size = 256

crop_size = 256

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    https://pytorch.org/hub/facebookresearch_pytorchvideo_slowfast/
    """
    def __init__(self):
        super().__init__()
        self.slowfast_alpha = 4
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // self.slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list


train_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ## 以下デバッグ用
                # transforms.Lambda(lambda x: [
                #     x, 
                #     print(type(x)),
                #     print(x.dtype),
                #     print(x.max()),
                #     print(x.min()),
                #     print(x.mean()),
                #     ]),
                # transforms.Lambda(lambda x: x[0]),
                RandomShortSideScale(min_size=256, max_size=320,),
                RandomCrop(256),
                RandomHorizontalFlip(),
                PackPathway(),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])

val_transform = Compose([
    ApplyTransformToKey(
        key="video",
        transform=Compose([
                UniformTemporalSubsample(args.video_num_subsampled),
                transforms.Lambda(lambda x: x / 255.),
                Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                ShortSideScale(256),
                CenterCrop(256),
                PackPathway(),
        ]),
    ),
    ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いておく
        transform=transforms.Lambda(lambda x: x - 1),
    ),
    RemoveKey("audio"),
])



2.56


In [5]:
root_UCF101 = '/mnt/HDD10TB/dataset/UFC101/'

train_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/trainlist01.txt',  # ラベルが1から101になっているので，transformで1を引いている
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=train_transform,
    )
val_set = Ucf101(
    data_path=root_UCF101 + 'ucfTrainTestlist/testlist01.txt',
    video_path_prefix=root_UCF101 + 'video/',
    clip_sampler=RandomClipSampler(clip_duration=clip_duration),
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=val_transform,
    )

num_classes = 101

In [6]:
train_set.num_videos

9537

In [7]:
val_set.num_videos

3783

In [8]:
# https://github.com/facebookresearch/pytorchvideo/blob/ef2d3a96bb939b12aa0f21fb467d2175b0f05e9f/tutorials/video_classification_example/train.py#L343
class LimitDataset(torch.utils.data.Dataset):
    """
    To ensure a constant number of samples are retrieved from the dataset we use this
    LimitDataset wrapper. This is necessary because several of the underlying videos
    may be corrupted while fetching or decoding, however, we always want the same
    number of steps per epoch.
    """

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

In [9]:
train_loader = DataLoader(LimitDataset(train_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)
val_loader = DataLoader(LimitDataset(val_set),
                            batch_size=args.batch_size,
                            drop_last=True,
                            num_workers=args.num_workers)


データローダのlenを確認．
- trainlist01.txtには9537行あるので「サンプル数＝ビデオ数」
- バッチサイズで割るとtrain_loaderのlengthになる

In [10]:
len(train_loader), train_set.num_videos, train_set.num_videos / args.batch_size

(596, 9537, 596.0625)

data loaderの挙動を確認．
- バッチはdictでやってくるので，`batch['video']`と`batch['label']`で取り出す
- RandomClipSamplerならランダムなラベルが得られている．

In [11]:
for i, batch in enumerate(train_loader):
    if i == 0:
        print(batch.keys())
        print(batch['video'][0].shape, batch['video'][1].shape)
    print(batch['label'].cpu().numpy())
    if i > 10:
        break

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
torch.Size([16, 3, 8, 256, 256]) torch.Size([16, 3, 32, 256, 256])
[25  3 24 85 84 85 23 54 11 43 15 36 12 88  4 15]
[ 69  48  61  52  58  69  33  63  24  35   2 100  34  89  39  10]
[38 59  6 33  9 69 31 14 17  7 85 79 29 42 53 47]
[98 77 84 41 79 38 22 84 96 81 17 51 42 34 38 91]
[84 58 60 24 53 40 20  1 10 57 26 53 52 88 34 78]
[84 88 12 19 75  5 91 70 41 47 74 81 96 25 10 29]
[17 99  2 64 70  0 69 65 39 48 70 99 37 17  6 93]
[85  7 38  1 33 53 16 85 33 70 87 10 21 91 94 24]
[87 50 59 16 44 91 91 17 62 11 36  2 47 21 86 67]
[ 76  87  33   4  99  23  87 100  34  66  16  76  44  84 100  48]
[58 10 70 87 67 27 37 20 53 92 19 73 18 60 13 29]
[ 1 72 67 74  8 38 54 85 11 15 49 80 37 77 30 86]


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

pytorchvideoのpretrained x3dモデルをダウンロード．
あとでsummaryを見れば分かるように，最終線形層は`model.blocks[6].proj`だからこれをnn.Linearに置き換える

- 注意：エラーが発生してダウンロードできない場合には，このnotebookの冒頭の注意書きを確認すること

In [13]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

# fine-tuningするなら以下を実行．スクラッチで学習するなら，実行しない
do_fine_tune = True
if do_fine_tune:
    for param in model.parameters():
        param.requires_grad = False

model.blocks[6].proj = nn.Linear(model.blocks[6].proj.in_features, num_classes)
model = model.to(device)

# data parallelだと性能が落ちる？
# model = nn.DataParallel(model)

Using cache found in /home/tamaki/.cache/torch/hub/facebookresearch_pytorchvideo_master


ランダムなデータを流し込んで出力されるかを確認する

In [14]:
data1 = torch.randn(2, 3, 8, 256, 256).to(device)
data2 = torch.randn(2, 3, 32, 256, 256).to(device)
model([data1, data2])

tensor([[ 0.1486,  0.1301,  0.0599, -0.0393, -0.2027,  0.2747, -0.2415, -0.0947,
         -0.0705, -0.1716,  0.2407,  0.1671, -0.0529,  0.1921, -0.1121, -0.0988,
         -0.1914, -0.0197,  0.0376, -0.0521,  0.1427,  0.0643,  0.0063, -0.0729,
          0.0035,  0.1370, -0.4055, -0.1332, -0.2426, -0.1637, -0.0376, -0.0058,
         -0.0630, -0.1669, -0.0143, -0.3453,  0.2857,  0.0489, -0.3149, -0.2889,
          0.0682,  0.0368,  0.0135,  0.3442,  0.0246, -0.1581,  0.2129,  0.2479,
         -0.0076, -0.3534, -0.1072, -0.1361,  0.2280,  0.1509,  0.0779,  0.1605,
         -0.1879, -0.0268,  0.2102,  0.0739, -0.2548, -0.0615, -0.0194,  0.1608,
         -0.1656,  0.0521, -0.1733,  0.1493, -0.0814, -0.1257,  0.1124, -0.1677,
         -0.1633, -0.1285, -0.1074,  0.1314, -0.1146, -0.1517,  0.2993, -0.1422,
         -0.0342, -0.3682,  0.0826, -0.0454,  0.0404, -0.1555, -0.0079, -0.0033,
         -0.0425,  0.2829,  0.1455,  0.0994,  0.1811, -0.1340, -0.0189, -0.2275,
          0.0391,  0.1711, -

summaryでinput/outputのサイズを確認できない．
sloffastは2つのtensorをリストで受け取るため．（なにか方法はあるか？）

In [15]:

torchinfo.summary(
    model.blocks if model.__class__.__name__ != 'DataParallel' else model.module.blocks,
    depth=4,
    # col_names=["input_size",
    #            "output_size"],
    col_names=["kernel_size", 
               "num_params"],
    row_settings=("var_names",)
    )

Layer (type (var_name))                                      Kernel Shape              Param #
ModuleList                                                   --                        --
├─MultiPathWayWithFuse (0)                                   --                        --
│    └─ModuleList (multipathway_blocks)                      --                        --
│    │    └─ResNetBasicStem (0)                              --                        --
│    │    │    └─Conv3d (conv)                               [3, 64, 1, 7, 7]          (9,408)
│    │    │    └─BatchNorm3d (norm)                          [64]                      (128)
│    │    │    └─ReLU (activation)                           --                        --
│    │    │    └─MaxPool3d (pool)                            --                        --
│    │    └─ResNetBasicStem (1)                              --                        --
│    │    │    └─Conv3d (conv)                               [3, 8, 5, 7, 7]           

便利関数を定義

In [16]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    https://github.com/machine-perception-robotics-group/attention_branch_network/blob/ced1d97303792ac6d56442571d71bb0572b3efd8/utils/misc.py#L59
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if type(val) == torch.Tensor:
            val = val.item()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def top1(outputs, targets):
    batch_size = outputs.size(0)
    _, predicted = outputs.max(1)
    return predicted.eq(targets).sum().item() / batch_size

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [35]:
num_epochs = 5

model = model.to(device)

with tqdm(range(num_epochs)) as pbar_epoch:
    for epoch in pbar_epoch:
        pbar_epoch.set_description("[Epoch %d]" % (epoch))


        with tqdm(enumerate(train_loader),
                  total=len(train_loader),
                  leave=True) as pbar_loss:

            train_loss = AverageMeter()
            train_acc = AverageMeter()
            model.train()

            for batch_idx, batch in pbar_loss:
                pbar_loss.set_description("[train]")

                targets = batch['label'].to(device)
                inputs = [b.to(device) for b in batch['video']]
                bs = inputs[0].size(0)  # current batch size, may vary at the end of the epoch

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss.update(loss, bs)
                train_acc.update(top1(outputs, targets), bs)

                pbar_loss.set_postfix_str(
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ' | loss={:6.04f} , top1={:6.04f}'
                    ''.format(
                    train_loss.avg, train_acc.avg,
                    train_loss.val, train_acc.val,
                ))



KeyboardInterrupt: 

fine-tuningなのでまあ速い．
- 4GPUでおよそ2.5it/s，1エポック約4分
- 1GPUでおよそ1.5it/s，1エポック約6分（596 iterations）


以下の設定
- video_num_subsampled = 32  # 32枚抜き出す      
- batch_size = 16
- num_workers = 24